In [6]:
import pandas as pd
from transformers import AutoTokenizer

In [7]:
!pip install amseg

  Preparing metadata (setup.py) ... done
  Created wheel for amseg: filename=amseg-2.3-py3-none-any.whl size=9773 sha256=9462583c9a4f6a6dfd3334cabecb4745a0fb86d8edfb6c6c5dd193afe2f0012b
  Stored in directory: /root/.cache/pip/wheels/df/9c/55/1368cec635d34f48f9ad4c7ae04d120b7acea9e371f3974d6f
Successfully built amseg


In [8]:
!pip install pdf2image

In [9]:
!pip install pytesseract

In [11]:
with open('/content/labeled_dataset_final.conll', 'r') as file:
    lines = file.readlines()

# Process lines as needed
data = [line.strip().split('\t') for line in lines]
df = pd.DataFrame(data)

In [ ]:
df

,0
0,3pcs B-PRODUCT
1,silicon I-PRODUCT
2,brush I-PRODUCT
3,spatulas I-PRODUCT
4,እስከ O
...,...
177806,ያሉበት O
177807,ድረስ O
177808,በነፃ O
177809,እናደርሳለን O


In [12]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [13]:
# data

In [14]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [15]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [16]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

In [17]:
# Display the first 10 results
for token, label in zip(aligned_tokens[:20], aligned_labels[:20]):
    print(f"{token:20} {label}")

▁Door                B-PRODUCT
▁Bot                 I-PRODUCT
tom                  O
▁Seal                I-PRODUCT
▁Strip               I-PRODUCT
▁Stop                I-PRODUCT
per                  O
▁ዋጋ                  B-PRICE
▁400                 I-PRICE
▁ብር                  O
▁ቁ                   B-LOC
1                    O
መ                    O
ገና                   O
ኛ                    O
▁መሰረት                I-LOC
▁ደ                   I-LOC
ፋር                   O
▁ሞ                   I-LOC
ል                    O


In [20]:
from amseg.amharicSegmenter import AmharicSegmenter

In [21]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [22]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [23]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [24]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [25]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")

Door: B-PRODUCT
Bottom: I-PRODUCT
Seal: I-PRODUCT
Strip: I-PRODUCT
Stopper: I-PRODUCT
ዋጋ: B-PRICE
400: I-PRICE
ብር: O
ቁ1መገናኛ: B-LOC
መሰረት: I-LOC
ደፋር: I-LOC
ሞል: I-LOC
ሁለተኛ: I-LOC
ፎቅ: I-LOC
ቢሮ: I-LOC
ቁ: I-LOC
S05S06: I-LOC
በበርዎ: O
ስር: O
አቧራ: O


In [26]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('/content/final_tokens_labels.csv', index=False)

In [27]:
# # Display results from index 50 to 80 with formatting
# for token, label in zip(aligned_tokens[:80], aligned_labels[:80]):
#     print(f"{token:<20} {label}")